In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.optimizers import Adam
from keras.layers import (Dropout, GlobalAveragePooling2D)
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import os
import csv
import optuna
from utils import *
from tensorflow.keras.applications import InceptionV3


In [4]:
metadata_path = "metadata.csv"
df = pd.read_csv(metadata_path)

In [6]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

In [14]:
train_ds = tf.data.TFRecordDataset(paths["train"])
train_ds = train_ds.map(parse_record, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(batch_size).prefetch(AUTOTUNE)

val_ds = tf.data.TFRecordDataset(paths["val"])
val_ds = val_ds.map(parse_record, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

test_ds = tf.data.TFRecordDataset(paths["test"])
test_ds = test_ds.map(parse_record, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

In [15]:
df['label'] = df['phylum'] + '_' + df['family']
class_names = sorted(df['label'].unique())
class_to_index = {name: idx for idx, name in enumerate(class_names)}
df['label_idx'] = df['label'].map(class_to_index)

In [16]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(class_names)),
    y=df['label_idx']
)

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

In [17]:
def create_model(trial):
    # Hiperparâmetros a testar
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    dense_units = trial.suggest_categorical("dense_units", [64, 128, 256])
    trainable_layers = trial.suggest_int("trainable_layers", 20, 100)

    # Base do modelo
    base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
    base_model.trainable = True

    # Congelar parte das camadas
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(dropout_rate),
        layers.Dense(dense_units, activation='relu'),
        layers.Dense(202, activation='softmax')  # 202 classes
    ])

    optimizer = optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

def objective(trial):
    model = create_model(trial)
    
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=20,
        class_weight=class_weights_dict,
        callbacks=[early_stop],
        verbose=0,
    )

    # Usa a melhor accuracy de validação como objetivo
    val_acc = max(history.history["val_accuracy"])
    return val_acc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # Faz 20 testes de configurações diferentes

# Resultados
print("Melhores hiperparâmetros:")
print(study.best_params)

[I 2025-04-26 01:17:49,188] A new study created in memory with name: no-name-ef69709c-8363-4896-955a-dd7d1dde165c
2025-04-26 01:20:10.576970: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
2025-04-26 01:20:41.694616: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-04-26 01:23:45.633388: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node It

Melhores hiperparâmetros:
{'learning_rate': 0.00022631443241333675, 'dropout_rate': 0.20862506393987143, 'dense_units': 256, 'trainable_layers': 71}
